In [78]:
import pandas as pd
import nltk as n
from collections import Counter as ctr
import itertools as i
import numpy as np

In [30]:
data = pd.read_csv('spam.csv',skiprows=1, names=['type','text', 'c3', 'c4', 'c5'], delimiter=',', encoding='ISO-8859-1')

In [31]:
data.drop(['c3','c4','c5'],axis=1,inplace=True)

In [46]:
data['clean'] = data.text.map(lambda x: x.lower().split())
data[:5]

,type,text,clean
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point,, crazy.., available..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar..., joking, wif, u, oni...]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor..., u, c, already..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, don't, think, he, goes, to, usf,, he,..."


In [47]:
ctr(data.type)

Counter({'ham': 4825, 'spam': 747})

In [48]:
test = data.sample(frac=0.1)
train = data[~data.index.isin(test.index)]

train.shape, test.shape

((5015, 3), (557, 3))

### Prior probability

In [49]:
ham_spam_ctr = ctr(train.type)

def Pa(X=''):
    return ham_spam_ctr[X] / len(train)

In [50]:
Pa(X='ham'), Pa(X='spam')

(0.8662013958125623, 0.1337986041874377)

### Normalization

In [95]:
words_ctr = ctr([word for row in train.clean for word in row])

def Pb(W=''):
    if W not in words_ctr: return 0.0001
    return words_ctr[W] / sum(words_ctr.values())

In [96]:
Pb(W='the')

0.015206781402517338

In [97]:
Pb(W='hello')

0.00023118417672745954

### Maximum Likelihood

In [107]:
words_ham_spam = {}

for col_name in list(set(data.type)):
    sub_df = train[train.type == col_name]
    words_ham_spam[col_name] = ctr([word for row in sub_df.clean for word in row])
    
def Pba(W='',X=''):
    part = words_ham_spam[X]
    if W not in part: return 0.000001
    return part[W] / sum(part.values())

In [108]:
Pba(W='the', X='ham')

0.01616031027795734

In [109]:
for part in words_ham_spam:
    words = words_ham_spam[part]
    print(part, round(sum([Pba(a,part) for a in words.keys()]),8))

ham 1.0
spam 1.0


In [110]:
def Pab(X='',W=''):
    return Pba(W,X) * Pa(X) / Pb(W)

In [111]:
Pab(X='ham', W='the')

0.9205158507252228

In [112]:
def Ps(X, T):
    return np.prod([Pab(X=X,W=word) for word in T])

In [113]:
Ps('spam', ['the','hello', 'ahoytherebigfella'])

9.818691354730446e-06

In [114]:
test['ham'] = test.clean.map(lambda x: Ps('ham',x))

In [123]:
test['spam'] = test.clean.map(lambda x: Ps('spam',x))

test.spam.describe()

count     5.570000e+02
mean      8.479467e-06
std       1.293111e-04
min      5.255107e-163
25%       7.446851e-33
50%       2.350540e-19
75%       3.470744e-13
max       2.604390e-03
Name: spam, dtype: float64

In [128]:
test['winner'] = test.ham / test.spam
test['winner'] = test.winner.map(lambda x: 'ham' if x>1 else 'spam')

In [133]:
result = test.winner == test.type

sum(result) / len(test)

0.9515260323159784